In [ ]:
%run reader.ipynb

In [ ]:
airs_data_asc = airs_read('AIRS/AIRS3STD_7_0_TotH2OVap_A.20160701-20220531.68W_76N_68W_76N.csv')
airs_data_disc = airs_read('AIRS/AIRS3STD_7_0_TotH2OVap_D.20160701-20220531.68W_76N_68W_76N.csv')
cimel_data = cimel_read('CIMEL/20170101_20211231_Thule.lev20')
ftir_data = ftir_read('FTIR/h2o_FTIR_2016_2020.dat')
mod_data = modis_read('MODIS/MOD05_5km_totale.csv')
myd_data = modis_read('MODIS/MYD05_5km_totale.csv')
radiosounding_data = radiosounding_read('RADIOSOUNDING/PWVradiosondeThule.txt')
vespa_data = vespa_read('VESPA/Vapor_20160716_20220311.txt')

In [ ]:
airs_data_disc

In [ ]:
year=[2016,2017,2018,2019]
gnss_data = pd.DataFrame()
hatpro_data = pd.DataFrame()

for year in year:
    gnss_data = gnss_data.append(gnss_read(
        f'GNSS/THU3_{year}global.txt',year))
    hatpro_data = hatpro_data.append(hatpro_read(
        f'HATPRO/IWV_{year}_HATPRO.txt',year))

### Drop rows with water values > 50mm 

In [ ]:
mod_data = df_drop(mod_data, mod_data.IWV_MODIS > 50.)
myd_data = df_drop(myd_data, myd_data.IWV_MODIS > 50. )

### Drop rows with distance values > 10 km 

In [ ]:
mod_data = distance_from_station(mod_data,drop_above=10.)
myd_data = distance_from_station(myd_data,drop_above=10.)

#### **MODIS and THAAO Integrated Water Vapor (IWV) timeseries**
##### Simple linear plots of the whole MODIS and THAAO IVW timeseries.

In [ ]:
width = 1200
height = 300 
size = 3
color_MOD = 'cyan'
color_MYD = '#F2583E'
color_THAAO = 'green'

defaults = dict(xlabel='Time', ylabel='IWV (mm)',height=height,
                width=width,alpha=100,dynspread=False,datashade=False,size=size, legend='top_right')

In [ ]:
MOD_timeseries = mod_data.IWV_MODIS.hvplot.scatter(label='IWV_MOD',cmap=[color_MOD],**defaults)
MYD_timeseries = myd_data.IWV_MODIS.hvplot.scatter(label='IWV_MYD',
                                                 color=color_MYD, cmap=[color_MYD],
                                                 **defaults)

airs_timeseries = airs_data.hvplot.scatter(label='AIRS',cmap=['orange'],
                                      color='orange',**defaults)

radiosounding_timeseries = radiosounding_data['IWV[mm]'].hvplot.scatter(label='Radiosounding',cmap=['black'],
                                                        color='black',**defaults)

hatpro_timeseries = hatpro_data['IWV[mm]'].hvplot.scatter(label='HATPRO',color='brown', cmap=['brown'],
                                         **defaults)

ftir_timeseries = ftir_data['IWV[mm]'].hvplot.scatter(cmap=['yellow'],label='FTIR',color='yellow',
                                     **defaults)

gnss_timeseries = gnss_data['IWV[mm]'].hvplot.scatter(cmap=['grey'],label='GNSS',color='grey',
                                     **defaults)

vespa_timeseries = vespa_data.IWV_THAAO.hvplot.scatter(color=color_THAAO, label='VESPA', 
                                        **defaults)

cimel_timeseries = cimel_data['IWV[mm]'].hvplot.scatter(color= 'purple', label = 'CIMEL',
                                       **defaults)

In [ ]:
gnss_timeseries*cimel_timeseries

In [ ]:
zz=(MOD_timeseries*MYD_timeseries 
 *airs_timeseries*radiosounding_timeseries
 *ftir_timeseries)#*hatpro_timeseries)

In [ ]:
zz*vespa_timeseries*gnss_timeseries*cimel_timeseries

In [ ]:
new_myd_vespa = closest_in_time2(myd_data,vespa_data,'IWV_MODIS','IWV_THAAO',on='Time',delta_time='10m')
new_mod_vespa = closest_in_time2(mod_data,vespa_data,'IWV_MODIS','IWV_THAAO',on='Time',delta_time='10m')

In [ ]:
#AIRS è su AQUA quindi confronto solo con myd

#new_mod_airs = closest_in_time2(mod_data,airs_data,'IWV_MODIS','IWV[mm]',on='Time')
new_myd_airs = closest_in_time2(myd_data,airs_data,'IWV_MODIS','IWV[mm]',on='Time',delta_time='24h')

In [ ]:
airs_data

In [ ]:
new_myd_vespa_timeseries = new_myd_vespa.IWV_MODIS.hvplot.scatter(x='Time_IWV_MODIS', 
                                                          label='IWV_MYD',
                                                          color=color_MYD, 
                                                          title= 'MYD-Thaao',**defaults)

new_mod_vespa_timeseries = new_mod_vespa.IWV_MODIS.hvplot.scatter(x='Time_IWV_MODIS', 
                                                          label='IWV_MOD',
                                                          color=color_MOD, 
                                                          title= 'MOD-Thaao',**defaults)

new_mod_airs_timeseries = new_mod_airs.IWV_MODIS.hvplot.scatter(x='Time_IWV_MODIS',
                                                               label='IWV_MOD',
                                                               title= 'MOD-AIRS',
                                                               **defaults)

new_myd_airs_timeseries = new_myd_airs.IWV_MODIS.hvplot.scatter(x='Time_IWV_MODIS',
                                                               label='IWV_MYD',
                                                               title= 'MYD-AIRS',
                                                               **defaults)


In [ ]:
(new_myd_vespa_timeseries*new_myd_vespa.IWV_THAAO.hvplot.scatter(x='Time_IWV_THAAO',
                                                                color=color_THAAO,
                                                               **defaults)
+new_mod_vespa_timeseries*new_mod_vespa.IWV_THAAO.hvplot.scatter(x='Time_IWV_THAAO',
                                                                color=color_THAAO,
                                                               **defaults)
+new_mod_airs_timeseries*new_mod_airs['IWV[mm]'].hvplot.scatter(x='Time_IWV[mm]',
                                                                color='orange',
                                                               **defaults)
+new_myd_airs_timeseries*new_myd_airs['IWV[mm]'].hvplot.scatter(x='Time_IWV[mm]',
                                                            color='orange',
                                                           **defaults)
).cols(1)

In [ ]:
slope = hv.Slope(1, 0).opts(line_color='red')

In [ ]:
new_mod_vespa = new_mod_vespa.dropna(subset=['IWV_MODIS','IWV_THAAO'])
new_myd_vespa = new_myd_vespa.dropna(subset=['IWV_MODIS','IWV_THAAO'])
new_modis_TOT_vespa = pd.concat([new_mod_vespa,new_myd_vespa], sort=True)

In [ ]:
regress_MOD_sk = regression_sklearn(new_mod_vespa.IWV_MODIS,new_mod_vespa.IWV_THAAO)
regress_MYD_sk = regression_sklearn(new_myd_vespa.IWV_MODIS,new_myd_vespa.IWV_THAAO)
regress_modis_TOT_sk = regression_sklearn(new_modis_TOT_vespa.IWV_MODIS,new_modis_TOT_vespa.IWV_THAAO)

In [ ]:
xlim = [0,5]
ylim = xlim
default2 = dict(x='IWV_MODIS', y='IWV_THAAO', height=300, width=1200, c='vza', 
                xlim=xlim, ylim=ylim,
                s='diff_distance', scale=3,
                ylabel='IWV THAAO (cm)', groupby=['Time.month'])
regress_default = dict(x_offset=xlim[0]-1.5,y_offset=ylim[0]+1.5)

regress_MOD_s_text = regression_eq_text(new_mod_vespa.IWV_MODIS,
                                        new_mod_vespa.IWV_THAAO, regress_MOD_sk,
                                        **regress_default)

regress_MYD_s_text = regression_eq_text(new_myd_vespa.IWV_MODIS,
                                         new_myd_vespa.IWV_THAAO, regress_MYD_sk,
                                        **regress_default)

regress_TOT_s_text = regression_eq_text(new_modis_TOT_vespa.IWV_MODIS,
                                        new_modis_TOT_vespa.IWV_THAAO, 
                                        regress_modis_TOT_sk,
                                        **regress_default)

ThaaoVsMOD_scatter = new_mod_vespa.hvplot.scatter(xlabel='IWV MOD (mm)',**default2)

ThaaoVsMYD_scatter = new_myd_vespa.hvplot.scatter(xlabel='IWV MYD (mm)', **default2)

In [ ]:
(ThaaoVsMOD_scatter*slope*regress_MOD_sk[0]*regress_MOD_s_text
 
+ ThaaoVsMYD_scatter*slope*regress_MYD_sk[0]*regress_MYD_s_text
 
+ (ThaaoVsMOD_scatter*ThaaoVsMYD_scatter*slope*regress_TOT_s_text
   *regress_MOD_sk[0]*regress_MYD_sk[0]*regress_modis_TOT_sk[0]
  ).opts(title='DDDDD', xlabel= 'IWV Modis Tot (mm)')).cols(1)

In [ ]:
new_mod_airs

In [ ]:
new_mod_airs = new_mod_airs.dropna(subset=['IWV_MODIS','IWV[mm]'])
new_myd_airs = new_myd_airs.dropna(subset=['IWV_MODIS','IWV[mm]'])
new_modis_TOT_airs = pd.concat([new_mod_airs,new_myd_airs], sort=True)

In [ ]:
regress_MOD_sk = regression_sklearn(new_mod_airs.IWV_MODIS,new_mod_airs.I)
regress_MYD_sk = regression_sklearn(new_myd_airs.IWV_MODIS,new_myd_airs.IWV_THAAO)
regress_modis_TOT_sk = regression_sklearn(new_modis_TOT_airs.IWV_MODIS,new_modis_TOT_airs.IWV_THAAO)

In [ ]:
xlim = [0,5]
ylim = xlim
default2 = dict(x='IWV_MODIS', y='IWV_THAAO', height=300, width=1200, c='vza', 
                xlim=xlim, ylim=ylim,
                s='diff_distance', scale=3,
                ylabel='IWV THAAO (cm)', groupby=['Time.month'])
regress_default = dict(x_offset=xlim[0]-1.5,y_offset=ylim[0]+1.5)

regress_MOD_s_text = regression_eq_text(new_mod_airs.IWV_MODIS,
                                        new_mod_airs.IWV_THAAO, regress_MOD_sk,
                                        **regress_default)

regress_MYD_s_text = regression_eq_text(new_myd_airs.IWV_MODIS,
                                         new_myd_airs.IWV_THAAO, regress_MYD_sk,
                                        **regress_default)

regress_TOT_s_text = regression_eq_text(new_modis_TOT_airs.IWV_MODIS,
                                        new_modis_TOT_airs.IWV_THAAO, 
                                        regress_modis_TOT_sk,
                                        **regress_default)

ThaaoVsMOD_scatter = new_mod_airs.hvplot.scatter(xlabel='IWV MOD (mm)',**default2)

ThaaoVsMYD_scatter = new_myd_airs.hvplot.scatter(xlabel='IWV MYD (mm)', **default2)

In [ ]:
(ThaaoVsMOD_scatter*slope*regress_MOD_sk[0]*regress_MOD_s_text
 
+ ThaaoVsMYD_scatter*slope*regress_MYD_sk[0]*regress_MYD_s_text
 
+ (ThaaoVsMOD_scatter*ThaaoVsMYD_scatter*slope*regress_TOT_s_text
   *regress_MOD_sk[0]*regress_MYD_sk[0]*regress_modis_TOT_sk[0]
  ).opts(title='DDDDD', xlabel= 'IWV Modis Tot (mm)')).cols(1)